# 観音台第二自治会 財政データ 可視化ノートブック

**対象期間**: 平成30年度（2018）〜 2024年度（7年間）  
**データソース**: 各年度総会資料（`fetch_financial_data.py` にて抽出・`financial_summary.md` にて確認済み）  
**目的**: 財政の「見える化」のための探索的可視化

---


## 1. ライブラリの読み込み


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib
import warnings
warnings.filterwarnings("ignore")

# --- 日本語フォント設定 ---
# Windows 環境では "MS Gothic" または "Yu Gothic" が利用可能
for font_name in ["Yu Gothic", "MS Gothic", "Meiryo", "IPAexGothic"]:
    try:
        matplotlib.font_manager.findfont(font_name, fallback_to_default=False)
        plt.rcParams["font.family"] = font_name
        print(f"日本語フォント設定: {font_name}")
        break
    except Exception:
        continue
else:
    print("⚠ 日本語フォントが見つかりません。文字化けする場合は font_name を手動設定してください。")

plt.rcParams["axes.unicode_minus"] = False
plt.rcParams["figure.dpi"] = 110

print(f"matplotlib: {matplotlib.__version__}")
print(f"pandas:     {pd.__version__}")


## 2. データ定義

総会資料から抽出・確認済みの決算実績値を直接定義します。  
（令和2〜4年度の会費・委託料内訳は追加確認が必要なため `None` としています）


In [ ]:
# ============================================================
# 一般会計 (決算実績値)
# ============================================================
#   収入合計 / うち自治会費 / うち委託料 / 支出合計
#   うち振替積立（集会所特別会計への振替）/ 前年繰越 / 次年繰越
# None = 原資料の当該箇所が未確認または表形式が異なる年度
# ============================================================
data_general = {
    "年度ラベル":  ["H30\n(2018)", "R1\n(2019)", "R2\n(2020)", "R3\n(2021)", "R4\n(2022)", "R5\n(2023)", "2024\n(2024)"],
    "西暦":        [2018, 2019, 2020, 2021, 2022, 2023, 2024],
    "収入合計":    [600931, 595184, 537074, 591013, 595429, 615065, 636699],
    "自治会費収入": [350250, 344500, None,   None,   None,   369000, None],
    "委託料収入":  [250660, 250660, None,   None,   None,   246050, None],
    "支出合計":    [314801, 343355, 226800, 209110, 166324, 1224350, 1276279],
    "振替積立":    [0,      0,      0,      0,      0,      1000000, 1000000],
    "実質支出":    [314801, 343355, 226800, 209110, 166324, 224350,  276279],
    "前年繰越":    [2306431, 2592561, 2844390, 3154664, 3536581, 3965686, 3356401],
    "次年繰越":    [2592561, 2844390, 3154664, 3536581, 3965686, 3356401, 2716821],
}

# ============================================================
# 集会所積立金特別会計 (決算実績値)
# ============================================================
data_special = {
    "西暦":       [2018, 2019, 2020, 2021, 2022, 2023, 2024],
    "収入合計":   [67662,  66512,  75012,  64812,  65062,  1064209, 1063610],
    "通常積立":   [67662,  66512,  75012,  64812,  65062,  64209,   63610],
    "一般振替入": [0,      0,      0,      0,      0,      1000000, 1000000],
    "支出合計":   [17397,  170279, 15610,  78802,  21471,  16098,   16857],
    "前年繰越":   [1478794, 1529059, 1425292, 1484694, 1470710, 1514301, 2562412],
    "次年繰越":   [1529059, 1425292, 1484694, 1470710, 1514301, 2562412, 3609165],
}

df_g = pd.DataFrame(data_general)
df_s = pd.DataFrame(data_special)

# 総資産テーブル（一般 + 特別 繰越合計）
df_total = pd.DataFrame({
    "西暦":       df_g["西暦"],
    "年度ラベル": df_g["年度ラベル"],
    "一般繰越":   df_g["次年繰越"],
    "特別繰越":   df_s["次年繰越"],
})
df_total["総計"] = df_total["一般繰越"] + df_total["特別繰越"]

print("=== 一般会計 ===")
print(df_g[["年度ラベル", "収入合計", "支出合計", "実質支出", "次年繰越"]].to_string(index=False))
print("\n=== 特別会計 ===")
print(df_s.to_string(index=False))
print("\n=== 総資産 ===")
print(df_total.to_string(index=False))


## 3. データ一覧表（整形表示）


In [ ]:
def fmt_yen(v):
    if pd.isna(v) or v is None:
        return "–"
    return f"{int(v):,}"

# 一般会計 サマリー表
cols_g = ["年度ラベル", "収入合計", "自治会費収入", "委託料収入", "支出合計", "振替積立", "実質支出", "次年繰越"]
display_df = df_g[cols_g].copy()
display_df.columns = ["年度", "収入合計", "うち会費", "うち委託料", "支出合計", "うち振替積立", "実質支出", "次年繰越"]
print("【一般会計 決算実績】（単位：円）")
display_df.style.format({
    "収入合計": fmt_yen,
    "うち会費": fmt_yen,
    "うち委託料": fmt_yen,
    "支出合計": fmt_yen,
    "うち振替積立": fmt_yen,
    "実質支出": fmt_yen,
    "次年繰越": fmt_yen,
}).set_caption("一般会計 決算実績（単位：円）").set_table_styles(
    [{"selector": "caption", "props": [("font-size", "14px"), ("font-weight", "bold")]}]
)


In [ ]:
# 特別会計 + 総資産 サマリー表
print("【特別会計 + 総資産 決算実績】（単位：円）")
df_combined = pd.DataFrame({
    "年度":     df_g["年度ラベル"],
    "特別収入": df_s["収入合計"],
    "うち通常積立": df_s["通常積立"],
    "うち振替入": df_s["一般振替入"],
    "特別支出": df_s["支出合計"],
    "特別繰越": df_s["次年繰越"],
    "一般繰越": df_g["次年繰越"],
    "総資産":   df_total["総計"],
})
df_combined.style.format({c: fmt_yen for c in df_combined.columns if c != "年度"}) \
    .set_caption("特別会計 + 総資産（単位：円）") \
    .set_table_styles(
        [{"selector": "caption", "props": [("font-size", "14px"), ("font-weight", "bold")]}]
    )


## 4. 折れ線グラフ：年度別推移（時系列）


In [ ]:
yr = df_g["西暦"]
labels = [l.replace("\n", "") for l in df_g["年度ラベル"]]

fig, axes = plt.subplots(3, 1, figsize=(10, 13), sharex=True)
fig.suptitle("観音台第二自治会 財政推移（7年間）", fontsize=15, fontweight="bold", y=1.01)

def add_yaxis_万円(ax):
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x/10000:.0f}万"))

# ---- (1) 総資産推移 ----
ax = axes[0]
ax.plot(yr, df_total["総計"],     "o-", color="#2563EB", lw=2.5, label="総計（一般+特別）")
ax.plot(yr, df_total["一般繰越"], "s--", color="#16A34A", lw=1.8, label="一般会計繰越")
ax.plot(yr, df_total["特別繰越"], "^--", color="#D97706", lw=1.8, label="特別会計繰越")
ax.set_title("総資産（繰越金）推移", fontsize=12)
ax.set_ylabel("金額（万円）")
ax.legend(loc="upper left", fontsize=9)
ax.grid(axis="y", alpha=0.4)
add_yaxis_万円(ax)
for x, y in zip(yr, df_total["総計"]):
    ax.annotate(f"{y/10000:.0f}万", (x, y), textcoords="offset points",
                xytext=(0, 8), ha="center", fontsize=8, color="#2563EB")

# ---- (2) 収入・実質支出（一般会計） ----
ax = axes[1]
ax.plot(yr, df_g["収入合計"],  "o-", color="#059669", lw=2.5, label="収入合計")
ax.plot(yr, df_g["実質支出"],  "s-", color="#DC2626", lw=2.5, label="実質支出（振替除く）")
ax.plot(yr, df_g["支出合計"],  "x:", color="#EF4444", lw=1.5, alpha=0.6, label="支出合計（振替込み）")
ax.set_title("一般会計：収入・支出推移", fontsize=12)
ax.set_ylabel("金額（万円）")
ax.legend(loc="upper left", fontsize=9)
ax.grid(axis="y", alpha=0.4)
add_yaxis_万円(ax)
for x, y in zip(yr, df_g["収入合計"]):
    ax.annotate(f"{y/10000:.1f}万", (x, y), textcoords="offset points",
                xytext=(0, 8), ha="center", fontsize=8, color="#059669")

# ---- (3) 特別会計推移 ----
ax = axes[2]
ax.plot(yr, df_s["次年繰越"],  "o-", color="#7C3AED", lw=2.5, label="特別会計繰越金")
ax.plot(yr, df_s["通常積立"],  "s--", color="#9CA3AF", lw=1.5, label="通常積立（会費50円×世帯）")
ax.bar(yr, df_s["一般振替入"], color="#DDD6FE", alpha=0.7, label="一般口座からの振替")
ax.plot(yr, df_s["支出合計"],  "^-", color="#F59E0B", lw=2.0, label="支出（修繕等）")
ax.set_title("集会所積立金特別会計推移", fontsize=12)
ax.set_ylabel("金額（万円）")
ax.set_xticks(yr)
ax.set_xticklabels(labels, fontsize=9)
ax.legend(loc="upper left", fontsize=9)
ax.grid(axis="y", alpha=0.4)
add_yaxis_万円(ax)

plt.tight_layout()
plt.savefig("financial_timeseries.png", dpi=130, bbox_inches="tight")
plt.show()
print("✓ 保存: financial_timeseries.png")


## 5. 棒グラフ：収入・支出の年度比較


In [ ]:
import numpy as np

fig, axes = plt.subplots(1, 2, figsize=(13, 6))
fig.suptitle("一般会計：収入・支出 年度比較", fontsize=14, fontweight="bold")

x = np.arange(len(yr))
w = 0.35

# ---- (左) 収入構成 積み上げ棒グラフ ----
ax = axes[0]
# 既知の内訳がある年度と不明な年度を分けて表示
会費_vals = [v if v is not None else 0 for v in df_g["自治会費収入"]]
委託_vals = [v if v is not None else 0 for v in df_g["委託料収入"]]
その他 = [r - f - c if f and c else r for r, f, c
          in zip(df_g["収入合計"], df_g["自治会費収入"], df_g["委託料収入"])]
その他_vals = [v if v is not None else 0 for v in その他]

b1 = ax.bar(x, 会費_vals,  width=0.6, label="自治会費", color="#4ADE80")
b2 = ax.bar(x, 委託_vals,  width=0.6, bottom=会費_vals,  label="委託料", color="#60A5FA")
b3 = ax.bar(x, その他_vals, width=0.6,
            bottom=[a+b for a,b in zip(会費_vals, 委託_vals)],
            label="その他・未分類", color="#D1D5DB")
ax.set_title("収入内訳（積み上げ）", fontsize=11)
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=8)
ax.set_ylabel("金額（万円）")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"{v/10000:.0f}万"))
ax.legend(fontsize=9)
ax.grid(axis="y", alpha=0.35)
for xi, rv in zip(x, df_g["収入合計"]):
    ax.text(xi, rv + 5000, f"{rv/10000:.1f}万", ha="center", fontsize=8, color="#1F2937")

# ---- (右) 収入 vs 実質支出 並列棒グラフ ----
ax = axes[1]
ax.bar(x - w/2, df_g["収入合計"],  w, label="収入合計",       color="#34D399")
ax.bar(x + w/2, df_g["実質支出"], w, label="実質支出（振替除く）", color="#F87171")
ax.set_title("収入 vs 実質支出（並列）", fontsize=11)
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=8)
ax.set_ylabel("金額（万円）")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"{v/10000:.0f}万"))
ax.legend(fontsize=9)
ax.grid(axis="y", alpha=0.35)

# 収支差額ラベル
for xi, r, e in zip(x, df_g["収入合計"], df_g["実質支出"]):
    diff = r - e
    ax.annotate(f"+{diff/10000:.1f}万", xy=(xi, max(r, e)),
                xytext=(0, 6), textcoords="offset points",
                ha="center", fontsize=8, color="#1D4ED8", fontweight="bold")

plt.tight_layout()
plt.savefig("financial_barchart.png", dpi=130, bbox_inches="tight")
plt.show()
print("✓ 保存: financial_barchart.png")


## 6. 積み上げ面グラフ：総資産の内訳推移

一般会計繰越（緑）+ 特別会計繰越（橙）= 総資産の積み上げ変化


In [ ]:
fig, ax = plt.subplots(figsize=(11, 6))

ax.stackplot(
    yr,
    df_total["一般繰越"],
    df_total["特別繰越"],
    labels=["一般会計 繰越金", "集会所積立金（特別会計）"],
    colors=["#86EFAC", "#FCD34D"],
    alpha=0.85,
)

# 総計の折れ線を重ねる
ax.plot(yr, df_total["総計"], "o-", color="#1E40AF", lw=2.5, label="総計", zorder=5)

# 各年の総計値ラベル
for x_, y_ in zip(yr, df_total["総計"]):
    ax.annotate(
        f"{y_/10000:.0f}万",
        (x_, y_),
        textcoords="offset points", xytext=(0, 10),
        ha="center", fontsize=9, fontweight="bold", color="#1E40AF"
    )

ax.set_title("総資産（繰越金）の内訳と推移", fontsize=14, fontweight="bold")
ax.set_xlabel("年度")
ax.set_ylabel("金額（万円）")
ax.set_xticks(yr)
ax.set_xticklabels(labels, fontsize=9)
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"{v/10000:.0f}万"))
ax.legend(loc="upper left", fontsize=10)
ax.grid(axis="y", alpha=0.4)

# 7年間の増加率を注釈
start = df_total["総計"].iloc[0]
end   = df_total["総計"].iloc[-1]
ax.annotate(
    f"7年間で +{(end-start)/10000:.0f}万円\n(+{(end-start)/start*100:.1f}%)",
    xy=(2024, end), xytext=(-90, -40),
    textcoords="offset points", fontsize=10, color="#DC2626",
    fontweight="bold",
    arrowprops=dict(arrowstyle="->", color="#DC2626"),
)

plt.tight_layout()
plt.savefig("financial_stacked_area.png", dpi=130, bbox_inches="tight")
plt.show()
print("✓ 保存: financial_stacked_area.png")


## 7. ヒートマップ：全指標の年度別ヒートマップ

全数値指標を正規化して色で比較します。明るい色 = 相対的に大きい値。


In [ ]:
try:
    import seaborn as sns
    HAS_SEABORN = True
except ImportError:
    HAS_SEABORN = False
    print("seaborn が未インストールのため、matplotlib でヒートマップを描画します。")

# ヒートマップ用データフレーム（None/NaN を含む全指標）
hm_data = {
    "一般:収入":   df_g["収入合計"].values,
    "一般:実質支出": df_g["実質支出"].values,
    "一般:振替積立": df_g["振替積立"].values,
    "一般:繰越":   df_g["次年繰越"].values,
    "特別:収入":   df_s["収入合計"].values,
    "特別:支出":   df_s["支出合計"].values,
    "特別:繰越":   df_s["次年繰越"].values,
    "総資産":      df_total["総計"].values,
}
df_hm = pd.DataFrame(hm_data, index=labels)

# 行ごと（指標ごと）に min-max 正規化
df_hm_norm = df_hm.T.copy().astype(float)
df_hm_norm = df_hm_norm.apply(lambda row: (row - row.min()) / (row.max() - row.min()), axis=1)

fig, ax = plt.subplots(figsize=(11, 6))
if HAS_SEABORN:
    sns.heatmap(
        df_hm_norm,
        ax=ax,
        cmap="YlGnBu",
        annot=df_hm.T.applymap(lambda v: f"{v/10000:.0f}万" if pd.notna(v) else "–"),
        fmt="",
        linewidths=0.5,
        linecolor="#E5E7EB",
        cbar_kws={"label": "正規化値（0=最小, 1=最大）"},
    )
else:
    im = ax.imshow(df_hm_norm.values, cmap="YlGnBu", aspect="auto")
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels, fontsize=9)
    ax.set_yticks(range(len(df_hm_norm.index)))
    ax.set_yticklabels(df_hm_norm.index, fontsize=9)
    for i in range(df_hm_norm.shape[0]):
        for j in range(df_hm_norm.shape[1]):
            raw = df_hm.T.iloc[i, j]
            text = f"{raw/10000:.0f}万" if pd.notna(raw) else "–"
            ax.text(j, i, text, ha="center", va="center", fontsize=8)
    plt.colorbar(im, ax=ax, label="正規化値（0=最小, 1=最大）")

ax.set_title("全財政指標 ヒートマップ（各指標内で年度比較）", fontsize=13, fontweight="bold")
plt.tight_layout()
plt.savefig("financial_heatmap.png", dpi=130, bbox_inches="tight")
plt.show()
print("✓ 保存: financial_heatmap.png")


## 8. インタラクティブウィジェット：表示する指標を選択


In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    HAS_WIDGETS = True
except ImportError:
    HAS_WIDGETS = False
    print("ipywidgets が未インストールのため、ウィジェットは無効です。")

if HAS_WIDGETS:
    # 選択できる指標と対応するデータ
    metric_options = {
        "一般会計：収入合計":     (yr, df_g["収入合計"],    "#059669", "万円"),
        "一般会計：実質支出":     (yr, df_g["実質支出"],    "#DC2626", "万円"),
        "一般会計：次年繰越":     (yr, df_g["次年繰越"],    "#2563EB", "万円"),
        "特別会計：収入合計":     (yr, df_s["収入合計"],    "#D97706", "万円"),
        "特別会計：支出合計":     (yr, df_s["支出合計"],    "#F59E0B", "万円"),
        "特別会計：次年繰越":     (yr, df_s["次年繰越"],    "#7C3AED", "万円"),
        "総資産（一般+特別合計）": (yr, df_total["総計"],   "#1E40AF", "万円"),
    }

    dropdown = widgets.Dropdown(
        options=list(metric_options.keys()),
        value="総資産（一般+特別合計）",
        description="指標:",
        style={"description_width": "60px"},
        layout=widgets.Layout(width="350px"),
    )
    out = widgets.Output()

    def update_chart(change):
        with out:
            clear_output(wait=True)
            key = dropdown.value
            x_data, y_data, color, unit = metric_options[key]
            fig, ax = plt.subplots(figsize=(10, 5))
            ax.bar(x_data, y_data, color=color, alpha=0.75, width=0.6)
            ax.plot(x_data, y_data, "o-", color=color, lw=2, zorder=5)
            for xi, yi in zip(x_data, y_data):
                if pd.notna(yi):
                    ax.text(xi, yi + max(y_data)*0.01, f"{yi/10000:.1f}万",
                            ha="center", fontsize=9, fontweight="bold", color=color)
            ax.set_title(key, fontsize=13, fontweight="bold")
            ax.set_xticks(x_data)
            ax.set_xticklabels(labels, fontsize=9)
            ax.set_ylabel(f"金額（{unit}）")
            ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"{v/10000:.0f}万"))
            ax.grid(axis="y", alpha=0.4)
            plt.tight_layout()
            plt.show()

    dropdown.observe(update_chart, names="value")
    display(widgets.VBox([dropdown, out]))
    update_chart(None)  # 初期表示
else:
    # フォールバック: 全指標を静的プロット
    all_metrics = {
        "一般：収入": df_g["収入合計"],
        "一般：実質支出": df_g["実質支出"],
        "一般：繰越": df_g["次年繰越"],
        "特別：繰越": df_s["次年繰越"],
        "総資産": df_total["総計"],
    }
    colors = ["#059669","#DC2626","#2563EB","#7C3AED","#1E40AF"]
    fig, axes = plt.subplots(1, len(all_metrics), figsize=(15, 5))
    for ax, (title, data), color in zip(axes, all_metrics.items(), colors):
        ax.bar(yr, data, color=color, alpha=0.75, width=0.5)
        ax.set_title(title, fontsize=9)
        ax.set_xticks(yr)
        ax.set_xticklabels([str(y) for y in yr], rotation=45, fontsize=7)
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"{v/10000:.0f}万"))
        ax.grid(axis="y", alpha=0.3)
    plt.suptitle("全指標 一覧", fontsize=12, fontweight="bold")
    plt.tight_layout()
    plt.show()


---

## 備考・次のステップ

| 状態 | 内容 |
|------|------|
| ✅ 確認済み | H30, R1, R5, 2024 の収入内訳（会費・委託料） |
| 🔲 要確認 | R2〜R4 の自治会費・委託料内訳（総会資料で確認可能） |
| 🔲 検討中 | 収支差額の折れ線グラフ追加 |
| 🔲 検討中 | 年報第1部後半への組み込み用グラフ選定 |
| 🔲 検討中 | 将来予測（繰越金の見通し）の追加 |

グラフ画像は `tools/` フォルダに PNG で保存されます。


## 9. 財政予測シミュレーション（30年）

### モデルの前提
- **開始点**: 2024年度末残高（一般繰越 2,716,821円・特別繰越 3,609,165円）
- **支出**: 実質支出 276,279円/年（固定）＋一般→特別会計振替 1,000,000円/年
- **特別会計支出**: 16,857円/年（固定）＋修繕費（指定年に一括）
- **シナリオ**: A・B・C の3案を比較


### 収入モデル（シナリオ別）> 集会所維持管理費（特別会計）への充当可否は別途検討が必要です。

> ⚠️ **備考**: ゴミ集積所使用料は一般会計収入として計上しますが、

| | 改定前（共通） | A・B（改定後） | C（改定後） |

|---|---|---|---|| 業務委託料 | 250,000円/年 | **2,400円/世帯分を減額** | **満額保証** |

| 会費対象 | 全世帯（正会員相当） | 普通＋優遇会員のみ | 全世帯（普通/優遇/賛助） || 非会員の収入 | — | ゴミ集積所使用料 12,000円/世帯 | — |
| 賛助比率相当の世帯 | 全員正会員扱い | **非会員** | **賛助会員**（加入） |

In [ ]:
# ============================================================
# 9-B. シミュレーション パラメータ定義
# ============================================================

# --- 会費シナリオ定義（冬アンケート3案） ---
SCENARIOS = [
    {
        "name":  "A: 現状維持（3,600円）",
        "年額":  3_600,
        "color": "gray",
        "style": "--",
    },
    {
        "name":  "B: 簡易化重視（3,000円）",
        "年額":  3_000,
        "color": "steelblue",
        "style": "-.",
    },
    {
        "name":  "C: 多様化許容（普通3,600/優遇1,200/賛助6,000円）",
        "年額":  None,   # 内訳で計算
        "内訳":  {"普通": 3_600, "優遇": 1_200, "賛助": 6_000},
        "比率":  {"普通": 0.50,  "優遇": 0.40,  "賛助": 0.10},
        "color": "green",
        "style": "-",
    },
]

# --- 固定パラメータ（デフォルト値） ---
REFORM_YEAR         = 2027        # 会費改定開始年
HOUSEHOLDS          = 113         # 総世帯数
COMMISSION          = 250_000     # 業務委託料・満額（円/年）
COMMISSION_PER_HH   = 2_400       # 非会員1世帯あたりの委託料減額分（円/年）
GARBAGE_FEE_PER_HH  = 12_000      # 非会員1世帯あたりのゴミ集積所使用料（円/年）
G_EXPENSE           = 276_279     # 一般会計実質支出（円/年）
TRANSFER            = 1_000_000   # 一般→特別 年間振替積立（円/年）
S_DEPOSIT           = 63_610      # 特別会計 通常積立（円/年）
S_EXPENSE           = 16_857      # 特別会計 通常支出（円/年）

# --- 修繕計画 ---
REPAIR_YEAR   = 2045
REPAIR_COST   = 10_000_000    # 1,000万円

# --- シミュレーション開始点（2024年度末残高） ---
INIT_G  = 2_716_821
INIT_S  = 3_609_165
HIST_TOTAL = list(zip(df_total["西暦"], df_total["総計"]))   # 過去実績

SIM_START = 2025
SIM_END   = 2055

print("パラメータ定義 完了")
print(f"  改定年: {REFORM_YEAR}年  修繕年: {REPAIR_YEAR}年  修繕費: {REPAIR_COST:,}円")

print(f"  初期残高: 一般 {INIT_G:,}円  特別 {INIT_S:,}円  合計 {INIT_G+INIT_S:,}円")

In [ ]:
# ============================================================
# 9-C. シミュレーション関数
# ============================================================

def simulate(scenario,
             households=HOUSEHOLDS,
             reform_year=REFORM_YEAR,
             repair_year=REPAIR_YEAR,
             repair_cost=REPAIR_COST,
             extra_saving_after_repair=True,
             normal_ratio=0.50,
             discount_ratio=0.40,
             garbage_usable=True):
    """
    30年財政シミュレーション。

    Parameters
    ----------
    scenario              : SCENARIOS リストの要素
    households            : 総世帯数
    reform_year           : 会費改定開始年
    repair_year           : 修繕実施年
    repair_cost           : 修繕費（円）
    extra_saving_after_repair : 修繕後も年間振替積立を継続するか
    normal_ratio          : 普通会員比率 (0–1)  ※シナリオA/Bでは「会員比率」としても使用
    discount_ratio        : 優遇会員比率 (0–1)
    garbage_usable        : ゴミ集積所使用料を一般会計収入（集会所維持管理費）に充当できるか

    Returns
    -------
    pd.DataFrame : columns = [西暦, 一般繰越, 特別繰越, 総計, ゴミ累計]
    """
    sanjo_ratio = max(0.0, 1.0 - normal_ratio - discount_ratio)
    # ^ 賛助比率 = シナリオCでは賛助会員、シナリオA/Bでは非会員（ゴミ使用料支払い）に対応

    g_bal         = INIT_G
    s_bal         = INIT_S
    garbage_cumul = 0      # ゴミ集積所使用料の累計額（充当可否にかかわらず常に記録）
    records       = []

    for year in range(SIM_START, SIM_END):
        # --- 会費・非会員収入の計算 ---
        if year < reform_year:
            # 改定前：全世帯が現行正会員扱い（3,600円）、委託料満額
            fee            = households * 3_600
            garbage_income = 0
            commission     = COMMISSION

        elif scenario["年額"] is not None:
            # シナリオ A / B（改定後）
            # 賛助比率相当の世帯 → 非会員（ゴミ集積所使用料を払い、委託料が減る）
            non_member_hh  = round(households * sanjo_ratio)
            member_hh      = households - non_member_hh
            fee            = member_hh * scenario["年額"]
            garbage_income = non_member_hh * GARBAGE_FEE_PER_HH if garbage_usable else 0
            # garbage_usable=True : 集積所使用料を集会所維持管理費に充当できる
            # garbage_usable=False: 集積所使用料は別途管理し、一般会計収入にはカウントしない
            commission     = max(0, COMMISSION - non_member_hh * COMMISSION_PER_HH)

            # 充当可否にかかわらず集積所使用料の発生額を累計（参考値として常に記録）
            garbage_cumul += non_member_hh * GARBAGE_FEE_PER_HH

        else:
            # シナリオ C（多様化・全世帯何らかの形で加入）
            # 賛助会員も自治会に加入 → 委託料は全世帯分が保証される
            d   = scenario["内訳"]
            avg = (normal_ratio   * d["普通"] +
                   discount_ratio * d["優遇"] +
                   sanjo_ratio    * d["賛助"])
            fee            = households * avg
            garbage_income = 0
            commission     = COMMISSION  # 満額保証

        g_income = fee + garbage_income + commission

        # --- 振替積立（修繕後の継続可否） ---
        do_transfer = TRANSFER if (year < repair_year or extra_saving_after_repair) else 0

        # --- 一般会計 更新 ---
        g_expense = G_EXPENSE + do_transfer
        g_bal = g_bal + g_income - g_expense

        # --- 特別会計 更新 ---
        s_income  = S_DEPOSIT + do_transfer
        s_expense = S_EXPENSE + (repair_cost if year == repair_year else 0)
        s_bal = s_bal + s_income - s_expense

        records.append({
            "西暦":     year,
            "一般繰越": round(g_bal),
            "特別繰越": round(s_bal),
            "総計":     round(g_bal + s_bal),
            "ゴミ累計": round(garbage_cumul),
        })

    return pd.DataFrame(records)


# 動作確認（デフォルトパラメータで3シナリオ比較）
print("シナリオ比較（世帯113・賛助10%・改定2027年）")
print(f"{'シナリオ':<25} {'2045年末':>14} {'2054年末':>14}  ゴミ累計(2054)")
for sc in SCENARIOS:
    df_fc = simulate(sc, garbage_usable=False)
    v45 = df_fc.loc[df_fc['西暦']==2045, '総計'].values[0]
    v54 = df_fc.loc[df_fc['西暦']==2054, '総計'].values[0]
    gc  = df_fc.loc[df_fc['西暦']==2054, 'ゴミ累計'].values[0]
    print(f"{sc['name'][:25]}: {v45:>14,.0f}円  {v54:>14,.0f}円  {gc/10_000:.0f}万円")


In [ ]:
# ============================================================
# 9-D. 3シナリオ比較グラフ（過去実績＋30年予測）
# ============================================================
# ゴミ集積所使用料の充当可否を切り替える変数
# True  = 使用可（シナリオA/BのゴミFeeを一般収入に計上）
# False = 使用不可（ゴミFeeを除いて計算）
STATIC_GARBAGE_USABLE = True

fig, ax = plt.subplots(figsize=(13, 6))

# --- 過去実績（実線・黒） ---
hist_yrs  = [y for y, _ in HIST_TOTAL]
hist_vals = [v / 1_000_000 for _, v in HIST_TOTAL]
ax.plot(hist_yrs, hist_vals, color="black", linewidth=2.5,
        marker="o", markersize=5, label="過去実績（総資産）", zorder=5)

# --- 各シナリオ予測 ---
for sc in SCENARIOS:
    df_fc = simulate(sc, garbage_usable=STATIC_GARBAGE_USABLE)
    fc_yrs  = df_fc["西暦"].tolist()
    fc_vals = (df_fc["総計"] / 1_000_000).tolist()

    joined_yrs  = [hist_yrs[-1]] + fc_yrs
    joined_vals = [hist_vals[-1]] + fc_vals

    ax.plot(joined_yrs, joined_vals,
            color=sc["color"], linestyle=sc["style"], linewidth=2,
            label=sc["name"])

gc_label = "集積所使用料：充当可" if STATIC_GARBAGE_USABLE else "集積所使用料：充当不可"
ax.text(0.01, 0.01, gc_label, transform=ax.transAxes,
        fontsize=9, color="gray", va="bottom")

# --- 修繕年マーカー ---
ax.axvline(REPAIR_YEAR, color="red", linestyle=":", linewidth=1.5, alpha=0.7)
ax.text(REPAIR_YEAR + 0.3, ax.get_ylim()[0] * 0.95 if ax.get_ylim()[0] > 0 else 0.3,
        f"修繕 {REPAIR_YEAR}年\n({REPAIR_COST/10_000:.0f}万円)",
        color="red", fontsize=9, va="bottom")

# --- 改定年マーカー ---
ax.axvline(REFORM_YEAR, color="orange", linestyle=":", linewidth=1.5, alpha=0.8)
ax.text(REFORM_YEAR + 0.3, (ax.get_ylim()[1] if ax.get_ylim()[1] > 0 else 10) * 0.92,
        f"会費改定\n{REFORM_YEAR}年",
        color="darkorange", fontsize=9, va="top")

# --- ゼロライン ---
ax.axhline(0, color="black", linewidth=0.8, linestyle="-", alpha=0.4)

# --- 装飾 ---
ax.set_title("総資産（一般＋特別会計）30年予測 ― 会費改定シナリオ比較", fontsize=13, pad=12)
ax.set_xlabel("年度", fontsize=11)
ax.set_ylabel("総資産（百万円）", fontsize=11)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda v, _: f"{v:.1f}M"))
ax.legend(loc="upper left", fontsize=9)
ax.set_xlim(2018, 2054)
ax.grid(True, alpha=0.3)
fig.tight_layout()

save_path = "forecast_scenarios.png"
fig.savefig(save_path, dpi=150)
plt.show()
print(f"グラフ保存: {save_path}")


In [ ]:
# ============================================================
# 9-E. インタラクティブ シミュレーター
# ============================================================

if not HAS_WIDGETS:
    print("ipywidgets が利用できません。セル 9-D の静的グラフをご覧ください。")
else:
    import ipywidgets as widgets
    from IPython.display import display

    # --- ウィジェット定義 ---
    w_households = widgets.IntSlider(
        value=113, min=80, max=140, step=1,
        description="世帯数:", style={"description_width": "80px"},
        layout=widgets.Layout(width="420px"))

    w_reform = widgets.IntSlider(
        value=2027, min=2026, max=2035, step=1,
        description="改定開始年:", style={"description_width": "80px"},
        layout=widgets.Layout(width="420px"))

    w_repair_year = widgets.IntSlider(
        value=2045, min=2030, max=2055, step=1,
        description="修繕年:", style={"description_width": "80px"},
        layout=widgets.Layout(width="420px"))

    w_repair_cost = widgets.IntSlider(
        value=1000, min=100, max=3000, step=50,
        description="修繕費(万円):", style={"description_width": "90px"},
        layout=widgets.Layout(width="430px"))

    w_normal = widgets.IntSlider(
        value=50, min=0, max=100, step=5,
        description="C:普通比率(%):", style={"description_width": "100px"},
        layout=widgets.Layout(width="440px"))

    w_discount = widgets.IntSlider(
        value=40, min=0, max=100, step=5,
        description="C:優遇比率(%):", style={"description_width": "100px"},
        layout=widgets.Layout(width="440px"))

    w_extra = widgets.Checkbox(
        value=True, description="修繕後も積立継続",
        layout=widgets.Layout(width="250px"))

    w_garbage = widgets.Checkbox(
        value=True, description="集積所使用料：集会所維持費に充当可",
        layout=widgets.Layout(width="300px"))

    w_label_sanjo = widgets.Label(value="  賛助比率: 10%")

    def update_sanjo_label(change):
        r = max(0, 100 - w_normal.value - w_discount.value)
        w_label_sanjo.value = f"  賛助比率: {r}%"

    w_normal.observe(update_sanjo_label, names="value")
    w_discount.observe(update_sanjo_label, names="value")

    out_sim = widgets.Output()

    def draw_forecast(households, reform_year, repair_year, repair_cost_man,
                      normal_pct, discount_pct, extra_saving, garbage_usable):
        out_sim.clear_output(wait=True)
        repair_cost = repair_cost_man * 10_000
        normal_r   = normal_pct  / 100
        discount_r = discount_pct / 100

        hist_yrs  = [y for y, _ in HIST_TOTAL]
        hist_vals = [v / 1_000_000 for _, v in HIST_TOTAL]

        with out_sim:
            fig2, ax2 = plt.subplots(figsize=(13, 6))

            ax2.plot(hist_yrs, hist_vals, color="black", linewidth=2.5,
                     marker="o", markersize=5, label="過去実績（総資産）", zorder=5)

            for sc in SCENARIOS:
                df_fc = simulate(sc,
                                 households=households,
                                 reform_year=reform_year,
                                 repair_year=repair_year,
                                 repair_cost=repair_cost,
                                 extra_saving_after_repair=extra_saving,
                                 normal_ratio=normal_r,
                                 discount_ratio=discount_r,
                                 garbage_usable=garbage_usable)
                fc_yrs  = df_fc["西暦"].tolist()
                fc_vals = (df_fc["総計"] / 1_000_000).tolist()
                joined_yrs  = [hist_yrs[-1]] + fc_yrs
                joined_vals = [hist_vals[-1]] + fc_vals
                ax2.plot(joined_yrs, joined_vals,
                         color=sc["color"], linestyle=sc["style"], linewidth=2,
                         label=sc["name"])

            # 集積所使用料の充当可否を注記
            gc_label = "集積所使用料：充当可" if garbage_usable else "集積所使用料：充当不可"
            ax2.text(0.01, 0.01, gc_label, transform=ax2.transAxes,
                     fontsize=9, color="gray", va="bottom")

            # マーカー
            ymin2, ymax2 = ax2.get_ylim()
            ax2.axvline(repair_year, color="red", linestyle=":", linewidth=1.5, alpha=0.7)
            ax2.text(repair_year + 0.3, max(ymin2, 0) + 0.3,
                     f"修繕 {repair_year}年\n({repair_cost_man}万円)",
                     color="red", fontsize=9, va="bottom")
            ax2.axvline(reform_year, color="orange", linestyle=":", linewidth=1.5, alpha=0.8)
            ax2.text(reform_year + 0.3, ymax2 * 0.92,
                     f"会費改定\n{reform_year}年",
                     color="darkorange", fontsize=9, va="top")
            ax2.axhline(0, color="black", linewidth=0.8, alpha=0.4)

            sanjo_pct = max(0, 100 - normal_pct - discount_pct)
            gc_suffix = "使用料充当可" if garbage_usable else "使用料充当不可"
            ax2.set_title(
                f"総資産 30年予測 ― {households}世帯 / 改定{reform_year}年 / "
                f"修繕{repair_year}年({repair_cost_man}万) / "
                f"C比率: 普通{normal_pct}%・優遇{discount_pct}%・賛助{sanjo_pct}% / {gc_suffix}",
                fontsize=11, pad=10)
            ax2.set_xlabel("年度", fontsize=11)
            ax2.set_ylabel("総資産（百万円）", fontsize=11)
            ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda v, _: f"{v:.1f}M"))
            ax2.legend(loc="upper left", fontsize=9)
            ax2.set_xlim(2018, 2054)
            ax2.grid(True, alpha=0.3)
            fig2.tight_layout()
            plt.show()

    interactive_out = widgets.interactive_output(
        draw_forecast,
        {
            "households":      w_households,
            "reform_year":     w_reform,
            "repair_year":     w_repair_year,
            "repair_cost_man": w_repair_cost,
            "normal_pct":      w_normal,
            "discount_pct":    w_discount,
            "extra_saving":    w_extra,
            "garbage_usable":  w_garbage,
        })

    panel = widgets.VBox([
        widgets.HTML("<b>【シミュレーター】パラメータを変更するとグラフが自動更新されます</b>"),
        widgets.HBox([
            widgets.VBox([w_households, w_reform, w_repair_year, w_repair_cost,
                          w_extra, w_garbage]),
            widgets.VBox([w_normal, w_discount, w_label_sanjo]),
        ]),
        interactive_out,
    ])
    display(panel)


## 9-F. ゴミ集積所使用料 累計推移グラフ　─ 設計根拠と予測の前提

### 図の目的

「ゴミ集積所使用料を集会所維持管理費（特別会計）に**充当できない**」と仮定した場合に、  
今後30年間でどれだけの費用が**別途発生するか**を可視化する。  
非会員世帯数の増減パターンが将来の財政に与える影響を、3つのシナリオで比較検討する。

---

### 実績データ（2018〜2025年度末）

| 年度 | 会員数 | 非会員数 | 備考 |
|------|-------:|--------:|------|
| 2018 | 113 | 1 | |
| 2019 | 112 | 1 | |
| 2020 | 111 | 1 | |
| 2021 | 109 | 2 | |
| 2022 | 110 | 2 | |
| 2023 | 105 | 6 | 急増 |
| 2024 | 106 | 7 | |
| 2025 | 102 | 10 | 最新実績（本グラフ基準点） |

- 総世帯数：**113世帯**（固定）
- ゴミ集積所使用料：**12,000円／非会員世帯／年**
- 2018〜2025年の累計実績：**約120,000円（12万円）**

---

### 主要パラメータ（コードセル 9-B より）

| パラメータ | 値 | 意味 |
|---|---|---|
| `GARBAGE_FEE_PER_HH` | 12,000円/年 | 非会員1世帯あたりのゴミ集積所使用料 |
| `HOUSEHOLDS` | 113世帯 | 総世帯数（会費改定後も変わらない前提） |
| `REPAIR_YEAR` | 2045年 | 集会所修繕の想定年 |
| `REPAIR_COST` | 1,000万円 | 修繕費（一括払い） |
| `SIM_END` | 2055年 | シミュレーション終了年（2054年度末） |
| 上限世帯数 (`_MAX_NM`) | 56世帯 | 総世帯数の半数。これ以上は増加しない仮定 |

---

### 3つの予測シナリオ（下段グラフ・ゴミ累計）

#### シナリオA：現状維持（グレー破線）
- **前提**: 非会員数が 2025年の10世帯のまま固定される
- **根拠**: 会費改定後に脱退歯止め施策が功を奏し、これ以上増加しないという楽観的な仮定
- **2054年末累計**：約396万円

#### シナリオB：穏やかな加速（ゴールド破線）
- **前提**: 直近3年（2023・2024・2025年）の実績（6・7・10世帯）を線形フィットし、その傾きで外挿
  - 線形傾き：約 **+2世帯/年**
  - 上限（56世帯）に達した時点で固定
- **根拠**: 過去の加速傾向が継続するが、加速率は一定（線形）という中程度の仮定
- **2054年末累計**：約1,445万円

#### シナリオC：指数加速・最悪ケース（赤橙破線）
- **前提**: 指数関数的な増加を仮定
  - フィット起点を **2014年**（正式記録の10年前）と設定し、2014〜2017年を「1世帯が存在した」と外挿した上でフィット
    → 起点を早めることで成長率を抑制し、非現実的な急加速を回避
  - 成長率 **r ≈ 0.213**（≈ 年21%増）
  - モデル式：`非会員数 ≈ 0.58 × exp(0.213 × (年 - 2014))`
  - 上限（56世帯）到達：**2036年頃**
- **累計計算の方法**:
  - 2018〜2054年のモデル非会員数で積み上げた累計に、**2025年実績値（約12万円）にアンカーするオフセット**を加算
  - これにより、モデル曲線の形状を保ちつつ実績との整合を取っている
- **根拠**: 施策が功を奏さず、加速が継続した場合の感度分析用上限シナリオ
- **2054年末累計**：約1,657万円

---

### 下段グラフの軸構成

| 軸 | 色 | 内容 |
|---|---|---|
| **左軸（トマト色）** | 赤系 | 非会員数（棒グラフ＋折れ線予測2本） |
| **右軸（黒・グレー・橙・赤橙）** | 多色 | ゴミ使用料の累計額（万円） |

棒グラフが左側に集中するため非会員数を左軸に、  
予測累計が右側に向かって大きくなるため累計を右軸に割り当てている。

---

### 上段グラフとの関係

上段グラフ（総資産推移）は、以下の組み合わせで描画される：

- **3つの会費シナリオ**（A：現状維持3,600円 ／ B：簡易化3,000円 ／ C：多様化許容）
- **2つの充当仮定**（充当不可・充当可）
- **穏やかな加速（シナリオB）** の影響を組み込んだライン（△マーカー付き）

「充当不可」ラインが本グラフのメインメッセージ。  
「充当可・参考」（薄い点線）は比較のための上限として表示。

---

### 年報への活用メモ

- **主訴**: 非会員が増えるほど累計が膨らみ、30年で最悪ケース1,600万円超になる
- **感度の確認**: シナリオAとCで4倍以上の開きがある → 早期の施策が財政に大きく効く
- **修繕時期との重複**: 集会所修繕（2045年・1,000万円）と累積使用料負担が同時発生するリスクに注意
- **データ引用元**: 各年の会員数・非会員数は総会決算資料より


In [ ]:

# ============================================================
# 9-F. ゴミ集積所使用料 累計推移グラフ
#       （充当不可と仮定した場合に別途積み上がる額の可視化）
# ============================================================
import numpy as np

# --- 実績データ：会員数・非会員数（年度末時点） ---
ACTUAL_MEMBERSHIP = {
    2018: (113, 1), 2019: (112, 1), 2020: (111, 1),
    2021: (109, 2), 2022: (110, 2), 2023: (105, 6),
    2024: (106, 7), 2025: (102, 10),
}
act_yrs        = sorted(ACTUAL_MEMBERSHIP.keys())
act_members    = [ACTUAL_MEMBERSHIP[y][0] for y in act_yrs]
act_nonmembers = [ACTUAL_MEMBERSHIP[y][1] for y in act_yrs]

act_garbage_cumul = []
_cumul = 0
for y in act_yrs:
    _cumul += ACTUAL_MEMBERSHIP[y][1] * GARBAGE_FEE_PER_HH
    act_garbage_cumul.append(_cumul / 10_000)

act_last_yr   = max(act_yrs)
act_cumul_end = act_garbage_cumul[-1]
_MAX_NM       = HOUSEHOLDS // 2

# ============================================================
# シナリオA：現状維持（固定10世帯）
# ============================================================
_proj_yrs = list(range(act_last_yr + 1, SIM_END + 1))
_flat_nm   = [act_nonmembers[-1]] * len(_proj_yrs)

flat_gc_yrs = [act_last_yr] + _proj_yrs
_c = act_cumul_end
flat_gc_vals = [_c]
for nm in _flat_nm:
    _c += nm * GARBAGE_FEE_PER_HH / 10_000
    flat_gc_vals.append(_c)

flat_nm = _flat_nm  # alias

# ============================================================
# シナリオB：穏やかな加速（線形）
# ============================================================
_recent_yrs = [2023, 2024, 2025]
_recent_nm  = [ACTUAL_MEMBERSHIP[y][1] for y in _recent_yrs]
_lin_slope  = (_recent_nm[-1] - _recent_nm[0]) / (len(_recent_yrs) - 1)

mild_nm = []
_nm_prev = act_nonmembers[-1]
for _ in _proj_yrs:
    _nm_new = min(_nm_prev + _lin_slope, _MAX_NM)
    mild_nm.append(round(_nm_new))
    _nm_prev = _nm_new

mild_gc_yrs = [act_last_yr] + _proj_yrs
_c = act_cumul_end
mild_gc_vals = [_c]
for nm in mild_nm:
    _c += nm * GARBAGE_FEE_PER_HH / 10_000
    mild_gc_vals.append(_c)

# ============================================================
# シナリオC：指数加速（最悪ケース）
# ============================================================
_ACCEL_ASSUME_START = 2014
import math
_known_yrs = [2018, 2025]
_known_nm  = [1, 10]
_b_exp = math.log(_known_nm[1] / _known_nm[0]) / (_known_yrs[1] - _known_yrs[0])
_a_exp = _known_nm[0] / math.exp(_b_exp * (_known_yrs[0] - _ACCEL_ASSUME_START))

accel_nm = []
_cap_yr = None
for yr in _proj_yrs:
    val = _a_exp * math.exp(_b_exp * (yr - _ACCEL_ASSUME_START))
    if val >= _MAX_NM:
        val = _MAX_NM
        if _cap_yr is None:
            _cap_yr = yr
    accel_nm.append(round(val))

# 2018年起点でモデル曲線全体を描画
_accel_all_yrs = list(range(_ACCEL_ASSUME_START, SIM_END + 1))
_accel_all_nm  = []
for yr in _accel_all_yrs:
    val = _a_exp * math.exp(_b_exp * (yr - _ACCEL_ASSUME_START))
    _accel_all_nm.append(min(val, _MAX_NM))

accel_gc_yrs = [act_last_yr] + _proj_yrs
_c = act_cumul_end
accel_gc_vals = [_c]
for nm in accel_nm:
    _c += nm * GARBAGE_FEE_PER_HH / 10_000
    accel_gc_vals.append(_c)

_accel_gc_vals_from_2025 = [act_cumul_end]
_c2 = act_cumul_end
for nm in accel_nm:
    _c2 += nm * GARBAGE_FEE_PER_HH / 10_000
    _accel_gc_vals_from_2025.append(_c2)

_nm_axis_max = max(_MAX_NM * 1.15, 15)

print(f"【穏やかな加速】線形傾き: +{_lin_slope:.1f}世帯/年（直近3年実績ベース）")
print(f"  2030年予測: {mild_nm[4]}世帯  /  2040年予測: {mild_nm[14]}世帯")
print(f"【指数加速（最悪）】 ≈ {_a_exp:.2f} × exp({_b_exp:.3f} × (年-{_ACCEL_ASSUME_START}))")
print(f"  上限（{_MAX_NM}世帯）到達予測: {_cap_yr}年")

# --- 穏やかな加速：総資産シミュレーション用 ---
_mild_nm_dict = dict(zip(_proj_yrs, mild_nm))

def _sim_mild_total(sc):
    """穏やかな加速（年次変動する非会員数）で総資産をシミュレーション（充当不可固定）"""
    g_bal, s_bal = INIT_G, INIT_S
    recs = []
    for year in range(SIM_START, SIM_END):
        if year < REFORM_YEAR:
            fee, commission = HOUSEHOLDS * 3_600, COMMISSION
        elif sc["年額"] is not None:
            non_hh = _mild_nm_dict.get(year, act_nonmembers[-1])
            mem_hh = HOUSEHOLDS - non_hh
            fee = mem_hh * sc["年額"]
            commission = max(0, COMMISSION - non_hh * COMMISSION_PER_HH)
        else:
            d = sc["内訳"]
            fee = HOUSEHOLDS * (0.50 * d["普通"] + 0.40 * d["優遇"] + 0.10 * d["賛助"])
            commission = COMMISSION
        g_bal += fee + commission - G_EXPENSE - TRANSFER
        s_bal += S_DEPOSIT + TRANSFER - S_EXPENSE - (REPAIR_COST if year == REPAIR_YEAR else 0)
        recs.append({"西暦": year, "総計": round(g_bal + s_bal)})
    return pd.DataFrame(recs)

# ============================================================
# フォントサイズ設定
FS_TITLE=14; FS_LABEL=12; FS_LEGEND=10; FS_ANNOT=9
FS_BAR=10;   FS_NOTE=9;   FS_MARK=10
# ============================================================

fig, axes = plt.subplots(2, 1, figsize=(13, 10), sharex=True,
                         gridspec_kw={"height_ratios": [3, 2]})

ax_main = axes[0]
# 下段：左軸＝非会員数（棒グラフ集中エリア）、右軸＝ゴミ累計（予測が右に展開）
ax_hh = axes[1]        # 左軸：非会員数
ax_gc = ax_hh.twinx()  # 右軸：集積所使用料累計

hist_yrs  = [y for y, _ in HIST_TOTAL]
hist_vals = [v / 1_000_000 for _, v in HIST_TOTAL]

# --- 上段：総資産グラフ ---
ax_main.plot(hist_yrs, hist_vals, color="black", linewidth=2.5,
             marker="o", markersize=5, label="過去実績（総資産）", zorder=5)

for sc in SCENARIOS:
    df_no  = simulate(sc, garbage_usable=False)
    df_yes = simulate(sc, garbage_usable=True)
    yrs_no  = [hist_yrs[-1]] + df_no["西暦"].tolist()
    vals_no = [hist_vals[-1]] + (df_no["総計"] / 1_000_000).tolist()
    yrs_yes = [hist_yrs[-1]] + df_yes["西暦"].tolist()
    vals_yes = [hist_vals[-1]] + (df_yes["総計"] / 1_000_000).tolist()
    ax_main.plot(yrs_no,  vals_no,  color=sc["color"], linestyle=sc["style"],
                 linewidth=2.2, label=f"{sc['name']} (充当不可)")
    ax_main.plot(yrs_yes, vals_yes, color=sc["color"], linestyle=":",
                 linewidth=1.2, alpha=0.45, label=f"{sc['name']} (充当可・参考)")

for sc in SCENARIOS:
    if sc["年額"] is None:
        continue
    df_m   = _sim_mild_total(sc)
    yrs_m  = [hist_yrs[-1]] + df_m["西暦"].tolist()
    vals_m = [hist_vals[-1]] + (df_m["総計"] / 1_000_000).tolist()
    ax_main.plot(yrs_m, vals_m, color=sc["color"], linestyle=sc["style"],
                 linewidth=1.3, alpha=0.6, marker="^", markevery=5, markersize=4,
                 label=f"{sc['name']} (充当不可・穏加速)")

ax_main.axvline(REPAIR_YEAR, color="red", linestyle=":", linewidth=1.5, alpha=0.7)
ax_main.axhline(0, color="black", linewidth=0.8, alpha=0.4)
ax_main.text(REPAIR_YEAR + 0.3, ax_main.get_ylim()[1] * 0.95,
             f"大規模修繕を想定\n{REPAIR_YEAR}年", color="red", fontsize=FS_MARK, va="top")
ax_main.set_ylabel("総資産（百万円）", fontsize=FS_LABEL)
ax_main.yaxis.set_major_formatter(plt.FuncFormatter(lambda v, _: f"{v:.1f}M"))
handles_main, labels_main = ax_main.get_legend_handles_labels()
leg1 = ax_main.legend(handles_main[:6], labels_main[:6],
                      loc="upper left", fontsize=FS_LEGEND, ncol=2)
ax_main.add_artist(leg1)
ax_main.legend(handles_main[6:], labels_main[6:],
               loc="lower left", bbox_to_anchor=(0.25, 0.02),
               fontsize=FS_LEGEND, ncol=1)
ax_main.grid(True, alpha=0.3)

# --- 下段左軸：非会員数 ---
# 実績（棒グラフ）
ax_hh.bar(act_yrs, act_nonmembers, width=0.5,
          color="tomato", alpha=0.65, label="非会員数（実績）", zorder=3)
for y, v in zip(act_yrs, act_nonmembers):
    if v > 0:
        ax_hh.text(y, v + 0.5, str(v), ha="center", fontsize=FS_BAR,
                   color="darkred", fontweight="bold")

# 穏やかな加速（橙折れ線）：2025実績から予測へ
ax_hh.plot([act_last_yr] + _proj_yrs, [act_nonmembers[-1]] + mild_nm,
           color="goldenrod", linestyle=":", linewidth=1.8,
           label="穏やか加速（予測）", zorder=4)

# 指数加速：2018年からモデル曲線をそのまま表示（実績値に接続しない）
ax_hh.plot(_accel_all_yrs, _accel_all_nm,
           color="orangered", linestyle=":", linewidth=1.8,
           label=f"指数加速モデル（2014年起点）", zorder=4)

# 上限ライン with 注記
ax_hh.axhline(_MAX_NM, color="orangered", linestyle="-", linewidth=0.8, alpha=0.5)
if _cap_yr:
    ax_hh.annotate(
        f"上限 {_MAX_NM}世帯（総世帯{HOUSEHOLDS}の半数）\n"
        f"※最悪ケースの仮定上限。{_cap_yr}年頃到達。",
        xy=(_cap_yr, _MAX_NM),
        xytext=(_cap_yr + 2, _MAX_NM * 0.80),
        fontsize=FS_NOTE, color="orangered",
        arrowprops=dict(arrowstyle="->", color="orangered", lw=1.0),
        bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="orangered", alpha=0.85),
    )

ax_hh.set_ylabel("非会員数（世帯）", fontsize=FS_LABEL, color="tomato")
ax_hh.tick_params(axis="y", labelcolor="tomato", labelsize=FS_ANNOT)
ax_hh.set_ylim(0, _nm_axis_max)
ax_hh.set_xlabel("年度", fontsize=FS_LABEL)
ax_hh.set_xlim(2018, 2054)
ax_hh.grid(True, alpha=0.25)

# --- 下段右軸：集積所使用料累計 ---
ax_gc.plot(act_yrs, act_garbage_cumul, color="black", linewidth=2.5,
           marker="o", markersize=6, zorder=5, label=f"実績累計（〜{act_last_yr}）")

ax_gc.fill_between(flat_gc_yrs, flat_gc_vals, alpha=0.10, color="dimgray")
ax_gc.plot(flat_gc_yrs, flat_gc_vals, color="dimgray", linestyle="--",
           linewidth=2, label=f"横ばい（10世帯固定）→ {flat_gc_vals[-1]:.0f}万円")

ax_gc.fill_between(mild_gc_yrs, flat_gc_vals, mild_gc_vals, alpha=0.15, color="goldenrod")
ax_gc.plot(mild_gc_yrs, mild_gc_vals, color="goldenrod", linestyle="--",
           linewidth=2, label=f"穏やか加速（+{_lin_slope:.0f}世帯/年）→ {mild_gc_vals[-1]:.0f}万円")

ax_gc.fill_between(mild_gc_yrs, mild_gc_vals, _accel_gc_vals_from_2025, alpha=0.15, color="orangered")
ax_gc.plot(accel_gc_yrs, accel_gc_vals, color="orangered", linestyle="--",
           linewidth=2, label=f"指数加速（最悪ケース）→ {accel_gc_vals[-1]:.0f}万円")

ax_gc.set_ylabel("集積所使用料 累計（万円）", fontsize=FS_LABEL)
ax_gc.set_ylim(bottom=0)

# 凡例：左軸（非会員数）を upper left、右軸（累計）を center right
handles_hh, labels_hh = ax_hh.get_legend_handles_labels()
handles_gc, labels_gc = ax_gc.get_legend_handles_labels()
ax_hh.legend(handles_hh, labels_hh, loc="upper left",  fontsize=FS_LEGEND, columnspacing=1.0)
ax_gc.legend(handles_gc, labels_gc, loc="center left", fontsize=FS_LEGEND, columnspacing=1.0)

# タイトル：tight_layout でサブプロット間隔を自動調整後、
# subplots_adjust(top) でタイトル用余白を確保する
fig.tight_layout()
fig.subplots_adjust(top=0.91)
fig.suptitle(
    "「ゴミ集積所使用料を充当不可」とした場合の比較\n"
    "（上）総資産推移 ／ （下）集積所使用料累計 ＋ 非会員数推移",
    fontsize=FS_TITLE, fontweight="bold", y=0.97)

save_path = "forecast_garbage_cumulative.png"
fig.savefig(save_path, dpi=150, bbox_inches='tight')
plt.show()
print(f"グラフ保存: {save_path}")
print(f"\n【2054年末 累計集積所使用料 予測】")
print(f"  横ばい（10世帯固定）: {flat_gc_vals[-1]:.0f}万円")
print(f"  穏やかな加速（線形）  : {mild_gc_vals[-1]:.0f}万円")
print(f"  指数加速  （最悪）    : {accel_gc_vals[-1]:.0f}万円")
